<a href="https://colab.research.google.com/github/EmmaLimoli/big-data-project/blob/main/Big_data_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (129 kB/s)
Reading package lists... Done


In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Books").getOrCreate()

In [4]:
# import Sparkfiles to read the amazon link, like a csv. 
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [6]:
# count the number of rows
df.count()

3105520

In [7]:
# drop na
df.dropna().show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [8]:
# drop duplicates
df.dropDuplicates()

DataFrame[marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [9]:
# count the number of rows
df.count()


3105520

In [10]:
# import types
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [11]:
# check describe for the structure types
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [12]:
# check column names
df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [13]:
# transform the dataset to fit the schema file
# create table 1: review_id_table

books_transform = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
books_transform.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RQ58W7SMO911M|   12076615|0385730586|     122662979| 2005-10-14|
|  RF6IUKMGL8SF|   12703090|0811828964|      56191234| 2005-10-14|
|R1DOSHH6AI622S|   12257412|1844161560|     253182049| 2005-10-14|
| RATOTLA3OF70O|   50732546|0373836635|     348672532| 2005-10-14|
|R1TNWRKIVHVYOV|   51964897|0262181533|     598678717| 2005-10-14|
|R2F53LI9KK9MOY|   31048862|0316769487|     862964341| 2005-10-14|
|R1KJ6MB7MRSQFF|   53000124|0805076069|     145341889| 2005-10-14|
|R2XIM9LT335WHE|   29732693|1581603681|     640542054| 2005-10-14|
|R1VE0FQQ0QTQJN|   48541186|0300108834|     915891133| 2005-10-14|
|R1VKEE2NWSWDRU|   15280864|0446531081|     880645124| 2005-10-14|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [14]:
# import date use to_date for the day

from pyspark.sql.functions import year, month, to_date

final = books_transform.withColumn('review_date', to_date(books_transform.review_date, 'yyyy-MM-dd'))


In [15]:
# printSchema to check structure types
final.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: date (nullable = true)



In [16]:
# check date

final.select(final["review_date"]).show()

+-----------+
|review_date|
+-----------+
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
| 2005-10-14|
+-----------+
only showing top 20 rows



In [17]:
# change customer_id = int, product_parent = int, review_date = date
# put final_df below to link to postgres

final_df = final.selectExpr("cast(review_id as string) review_id", "cast(customer_id as int) customer_id", "cast(product_id as string) product_id", "cast(product_parent as int) product_parent", "cast(review_date as date) review_date")
final_df.printSchema()
final_df.show()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RQ58W7SMO911M|   12076615|0385730586|     122662979| 2005-10-14|
|  RF6IUKMGL8SF|   12703090|0811828964|      56191234| 2005-10-14|
|R1DOSHH6AI622S|   12257412|1844161560|     253182049| 2005-10-14|
| RATOTLA3OF70O|   50732546|0373836635|     348672532| 2005-10-14|
|R1TNWRKIVHVYOV|   51964897|0262181533|     598678717| 2005-10-14|
|R2F53LI9KK9MOY|   31048862|0316769487|     862964341| 2005-10-14|
|R1KJ6MB7MRSQFF|   53000124|0805076069|     145341889| 2005-10-14|
|R2XIM9LT335WHE|   29732693|1581603681|     640542054| 2005-10-14|
|R1VE0FQQ0QTQJN|   48541186|0300108834|   

In [18]:
# create table 2:products
# pull product_id and product title

books_transform_2 = df.select(["product_id", "product_title"])
books_transform_2.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0385730586|Sisterhood of the...|
|0811828964|The Bad Girl's Gu...|
|1844161560|Eisenhorn (A Warh...|
|0373836635|Colby Conspiracy ...|
|0262181533|The Psychology of...|
|0316769487|The Catcher in th...|
|0805076069|Bait and Switch: ...|
|1581603681|Opening Combinati...|
|0300108834|A Little History ...|
|0446531081|Hour Game (King &...|
+----------+--------------------+
only showing top 10 rows



In [19]:
# change structure types of match table 2

table_two = books_transform_2.selectExpr("cast(product_id as string) product_id", "cast(product_title as string) product_title")
table_two.printSchema()
table_two.show()


root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0385730586|Sisterhood of the...|
|0811828964|The Bad Girl's Gu...|
|1844161560|Eisenhorn (A Warh...|
|0373836635|Colby Conspiracy ...|
|0262181533|The Psychology of...|
|0316769487|The Catcher in th...|
|0805076069|Bait and Switch: ...|
|1581603681|Opening Combinati...|
|0300108834|A Little History ...|
|0446531081|Hour Game (King &...|
|0976822105|     Faith is a Verb|
|1580085695|Furry Logic: A Gu...|
|0029148510|Acts of War: Beha...|
|1592285570|Temple to the Win...|
|0811848833|The Star Wars Pos...|
|0782144276|Photoshop for Nat...|
|0375757996|The Basic Works o...|
|013146308X|The Pterosaurs: F...|
|0375701907|Straight Man: A N...|
|141378240X|   Lipstick's Legacy|
+----------+--------------------+
only showing top 20 rows



In [20]:
# prepare to create table 3
# pull customer_id from df original variable

customer_count = df.select(df["customer_id"]).show()

+-----------+
|customer_id|
+-----------+
|   12076615|
|   12703090|
|   12257412|
|   50732546|
|   51964897|
|   31048862|
|   53000124|
|   29732693|
|   48541186|
|   15280864|
|   24209213|
|   52253037|
|   27925116|
|   15005044|
|   52534781|
|   26217071|
|   37684582|
|   45232107|
|   15437121|
|   52157117|
+-----------+
only showing top 20 rows



In [21]:
# create customer count column for third table use groupBy

customer_table = df.groupBy("customer_id").count()
customer_table.show(10)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   52496855|    8|
|   12079026|    1|
|   49471923|    2|
|   40347396|   75|
|   12083267|    1|
|   25824372|    1|
|   29853725|    1|
|   12934400|    1|
|   51371372|    1|
|   37236260|   72|
+-----------+-----+
only showing top 10 rows



In [22]:
# rename count column to customer_count to match table name

customer_table.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- count: long (nullable = false)



In [27]:
# create table 3: customers
# showcase only these two columns

customer_df = customer_table.withColumnRenamed('count', 'customer_count')
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52496855|             8|
|   12079026|             1|
|   49471923|             2|
|   40347396|            75|
|   12083267|             1|
|   25824372|             1|
|   29853725|             1|
|   12934400|             1|
|   51371372|             1|
|   37236260|            72|
|   50649342|             2|
|   49379318|             3|
|   39614932|             2|
|   12462751|             1|
|   50018404|             1|
|   50860080|            27|
|   38825828|            11|
|   45193257|           671|
|   49161738|             1|
|   43470228|             5|
+-----------+--------------+
only showing top 20 rows



In [28]:
# change type structions, check schema to see if it changed

customer_final = customer_df.selectExpr("cast(customer_id as int) customer_id", "cast(customer_count as int) customer_count")
customer_final.printSchema()


root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [31]:
# create table 4: vine_table

vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RQ58W7SMO911M|          4|            2|          3|   N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|
|R1DOSHH6AI622S|          4|            1|         22|   N|
| RATOTLA3OF70O|          5|            2|          2|   N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|
|R2XIM9LT335WHE|          4|            3|         10|   N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [32]:
# change type structures for table 4

final_vine = vine_table.selectExpr("cast(review_id as string) review_id", "cast(star_rating as int) star_rating", "cast(helpful_votes as int) helpful_votes", "cast(total_votes as int) total_votes", "cast(vine as string) vine")
final_vine.printSchema()
final_vine.show(10)


root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RQ58W7SMO911M|          4|            2|          3|   N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|
|R1DOSHH6AI622S|          4|            1|         22|   N|
| RATOTLA3OF70O|          5|            2|          2|   N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|
|R2XIM9LT335WHE|          4|            3|         10|   N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|
+--

In [8]:
# load the dataframe into the RDS instance
# connect to postgres

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.czptynze0rvd.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table 1 review_id_table in RDS

final_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to table 2 products in RDS

table_two.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# table 3: customers

customer_final.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# table 4 vine_table

final_vine.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)